In [3]:
today_date = '2026-02-13'


StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 5, Finished, Available, Finished, False)

In [4]:
Fabric_bronze_path = 'abfss://Sales_workspace@onelake.dfs.fabric.microsoft.com/Sales_lakehouse.Lakehouse/Tables/dbo/tblsales_bronze'

from pyspark.sql.functions import col
df = spark.read.format('delta').load(Fabric_bronze_path).filter(col("Processed_date")==str(today_date))
display(df)


StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 6, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 91d89ea2-1583-4217-a4e6-8c36538aea34)

In [9]:
### Data cleaning 

### Handling Duplicates
print('Before removing Duplicates',df.count())
df_distinct = df.dropDuplicates()
print('After removing Duplicates',df.count())



StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 11, Finished, Available, Finished, False)

Before removing Duplicates 1675
After removing Duplicates 1675


In [10]:
### Data cleaning 

### Drop rows with missing values 

df_nonNull = df_distinct.dropna(subset=['Order_ID','Customer_ID'])


StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 12, Finished, Available, Finished, False)

In [14]:
## Business Transformation 
## calculating Delivery days


df_Dev_date = df_nonNull.withColumn("Delivery_Days",(col('Ship_date')-col('Order_Date')).cast('int'))

display(df_Dev_date)


StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 16, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 46c50a2f-79e3-4d9d-b56d-c108f02c8617)

In [15]:
df_profit_margin = df_Dev_date.withColumn("Profit_Margin",(col('Profit')/col('Sales')))
display(df_profit_margin)

StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 17, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, ceb96b85-db2d-439d-b083-d3858a353c5d)

****Writing to Silver Layer****

In [16]:
df_profit_margin.createOrReplaceTempView('t_tblsales_Silver')

StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 18, Finished, Available, Finished, False)

In [17]:
%%sql
select * from t_tblsales_Silver

StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 19, Finished, Available, Finished, False)

<Spark SQL result set with 1000 rows and 29 fields>

In [20]:
Fabric_tblSales_Silver = "abfss://Sales_workspace@onelake.dfs.fabric.microsoft.com/Sales_lakehouse.Lakehouse/Tables/dbo/tblsales_silver"
try:
    spark.read.format('delta').load(Fabric_tblSales_Silver).createOrReplaceTempView('tblsales_Silver')

except:
    v_create_table = f"""CREATE TABLE IF NOT EXISTS tblsales_Silver (
Row_ID    string ,
Order_ID    string ,
Order_Date   date ,
Ship_Date   date ,
Ship_Mode    string ,
Customer_ID    string ,
Customer_Name    string ,
Segment    string ,
Postal_Code    string ,
City    string ,    
State    string ,  
Country    string ,     
Region    string ,     
Market    string , 
Product_ID    string , 
Category    string ,     
Sub_Category    string ,  
Product_Name    string ,     
Sales  DOUBLE ,     
Quantity  int , 
Discount  DOUBLE , 
Profit  DOUBLE ,  
Shipping_Cost  DOUBLE , 
Order_Priority    string , 
Month    string , 
Year    string ,
processed_date date,
Delivery_Days int,
Profit_Margin DOUBLE
)"""

spark.sql(v_create_table)
spark.read.format('delta').load(Fabric_tblSales_Silver).createOrReplaceTempView('tblsales_Silver')


StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 29, Finished, Available, Finished, False)

In [21]:
%%sql
select * from tblsales_Silver

StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 30, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 29 fields>

In [22]:
sql_statement = f"""merge into tblsales_silver as target using 
                    t_tblsales_Silver as source 
                    on target.Order_ID = source.Order_ID and target.Customer_ID = source.Customer_ID
                    WHEN MATCHED THEN 
                    UPDATE SET 
                    target.Row_ID = source.Row_ID,
                    target.Order_ID = source.Order_ID,
                    target.Order_Date = source.Order_Date,
                    target.Ship_Date = source.Ship_Date,
                    target.Ship_Mode  = source.Ship_Mode ,
                    target.Customer_ID = source.Customer_ID,
                    target.Customer_Name = source.Customer_Name,
                    target.Segment = source.Segment,
                    target.Postal_Code  = source.Postal_Code ,
                    target.City = source.City,
                    target.State = source.State,
                    target.Country = source.Country,
                    target.Region = source.Region,
                    target.Market = source.Market,
                    target.Product_ID   = source.Product_ID  ,
                    target.Category = source.Category,
                    target.Sub_Category = source.Sub_Category,
                    target.Product_Name = source.Product_Name,
                    target.Sales = source.Sales,
                    target.Quantity = source.Quantity,
                    target.Discount = source.Discount,
                    target.Profit   = source.Profit  ,
                    target.Shipping_Cost = source.Shipping_Cost,
                    target.Order_Priority = source.Order_Priority,
                    target.Month = source.Month,
                    target.Year = source.Year,
                    target.processed_date = source.processed_date,
                    target.Delivery_Days = source.Delivery_Days,
                    target.Profit_Margin = source.Profit_Margin
                WHEN NOT MATCHED THEN 
                    INSERT (
                    Row_ID   ,   
                    Order_ID   ,   
                    Order_Date     ,
                    Ship_Date     ,
                    Ship_Mode    ,  
                    Customer_ID   ,   
                    Customer_Name  ,    
                    Segment      ,
                    Postal_Code   ,   
                    City          ,
                    State        ,
                    Country       ,    
                    Region         ,  
                    Market       ,
                    Product_ID    ,  
                    Category        ,   
                    Sub_Category   ,     
                    Product_Name    ,       
                    Sales     ,    
                    Quantity    , 
                    Discount     , 
                    Profit     ,  
                    Shipping_Cost     , 
                    Order_Priority   ,    
                    Month ,      
                    Year   ,   
                    processed_date   ,
                    Delivery_Days  ,
                    Profit_Margin
                           ) VALUES (
                           source.Row_ID  ,    
source.Order_ID      ,
source.Order_Date     ,
source.Ship_Date     ,
source.Ship_Mode      ,
source.Customer_ID     , 
source.Customer_Name    ,  
source.Segment      ,
source.Postal_Code   ,   
source.City          ,
source.State        ,
source.Country       ,    
source.Region         ,  
source.Market       ,
source.Product_ID    ,   
source.Category       ,   
source.Sub_Category     ,   
source.Product_Name    ,       
source.Sales     ,     
source.Quantity    , 
source.Discount     , 
source.Profit     ,  
source.Shipping_Cost     , 
source.Order_Priority,       
source.Month      , 
source.Year      ,
source.processed_date ,
source.Delivery_Days ,
source.Profit_Margin
)"""
spark.sql(sql_statement).show()                

StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 31, Finished, Available, Finished, False)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|             1675|               0|               0|             1675|
+-----------------+----------------+----------------+-----------------+



In [23]:
%%sql
select * from tblsales_silver

StatementMeta(, 486f40ec-328a-4d94-8437-d637d5cd0bf8, 32, Finished, Available, Finished, False)

<Spark SQL result set with 1000 rows and 29 fields>